In [2]:
import numpy as np
from random import shuffle
import matplotlib.pyplot as plt
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
cvxopt_solvers.options['show_progress'] = False

#Polynomial Kernel
def kernel(x, y, order):
    return (1 + np.dot(x, y)) ** order

#Training Function
def train(X, y, classes, C, order):
  m,n = X.shape
  params = {}

  #Kernel Matrix
  H = np.zeros((m,m))
  for i in range(m):
    for j in range(m):
      H[i,j] = kernel(X[i],X[j],order)
  print("H Matrix done")
  
  for i in range(classes):
    #CVXOPT Initialization
    yc = y/(i+1)
    yc[yc != 1] = -1
    #print(yc[:200])
    P = cvxopt_matrix(np.outer(yc,yc) * H)
    q = cvxopt_matrix(-np.ones((m, 1)))
    G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
    h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
    A = cvxopt_matrix(yc.reshape(1, -1))
    b = cvxopt_matrix(np.zeros(1))

    #Run Solver
    svm_h = cvxopt_solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(svm_h['x'])

    #All SVs
    S = (alphas > 1e-9)
    y_sv = yc[S]
    alphas_sv = alphas[S]
    X_sv = X[S]
    #print(len(alphas_sv))

    #Marginal SVs
    T = ((alphas > 1e-9) &(alphas < (C-1e-4)))
    y_msv = yc[T]
    alphas_msv = alphas[T]
    X_msv = X[T]
    #print(len(alphas_msv))

    #bias calculation
    b = 0
    for a in range(len(alphas_msv)):
      sum = 0
      for s in range(len(alphas_sv)):
        sum = sum + alphas_sv[s] * y_sv[s] * kernel(X_msv[a],X_sv[s],order)
      b = b + y_msv[a] - sum
    b = b/len(alphas_msv)

    #Parameters for class i+1
    params['alphas_sv' + str(i+1)] = alphas_sv
    params['X_sv' + str(i+1)] = X_sv
    params['y_sv' + str(i+1)] = y_sv
    params['b' + str(i+1)] = b

    print("Number of Classes Completed:", i+1)

  return(params)

#Prediction Function
def predict(X, params, classes, order):
  score_set = np.zeros((classes)) #

  for i in range(classes):
    alphas_sv = params['alphas_sv' + str(i+1)]
    X_sv = params['X_sv' + str(i+1)]
    y_sv = params['y_sv' + str(i+1)]
    b = params['b' + str(i+1)]

    sum = 0
    for s in range(len(alphas_sv)):
      sum += alphas_sv[s] * y_sv[s] * kernel(X, X_sv[s],order)
    sum += b
    score_set[i] = sum

  prediction = np.argmax(score_set) + 1.
  return(prediction)

In [ ]:
#Test Data
X_train = np.load('../input/svmemg-data/X_train.npy')
Y_train = np.load('../input/svmemg-data/Y_train.npy')
X_test = np.load('../input/svmemg-data/X_test2.npy')
Y_test = np.load('../input/svmemg-data/Y_test2.npy')
index = [i for i in range(len(X_train))]
shuffle(index)
X_train = X_train[index]
Y_train = Y_train[index]
index = [i for i in range(len(X_test))]
shuffle(index)
X_test = X_test[index]
Y_test = Y_test[index]
X = np.vstack((X_train[:12000],X_test[:2000]))
Y = np.hstack((Y_train[:12000],Y_test[:2000]))
Y  = Y + 1

classes = 8
C = 5

#Training
for order in [4]: #test for order of polynomial kernel
  parameters = train(X,Y,classes,C,order)
  fname = "{0}{1}.npy".format("./5parameters", order)
  np.save(fname,parameters)

#Test Data 2

Y_test  = Y_test + 1

for order in [4]:
  fname = "{0}{1}.npy".format("./5parameters", order)
  print(fname)
  parameters = np.load(fname,allow_pickle='TRUE').item()
  print(parameters['alphas_sv1'])

  #Prediction
  y_predict = np.zeros(len(X_test))
  for i in range(len(X_test)):
    y_predict[i] = predict(X_test[i],parameters,classes,order)

  correct = np.sum(y_predict == Y_test)
  print(correct, "are correct out of", len(y_predict),"test samples")
  print("Accuracy =",(correct*100)/len(y_predict),"%")

#Test Data
X_test = np.load('../input/svmemg-data/X_test.npy')
Y_test = np.load('../input/svmemg-data/Y_test.npy')
Y_test  = Y_test + 1

for order in [4]:
  fname = "{0}{1}.npy".format("./5parameters", order)
  print(fname)
  parameters = np.load(fname,allow_pickle='TRUE').item()
  print(parameters['alphas_sv1'])

  #Prediction
  y_predict = np.zeros(len(X_test))
  for i in range(len(X_test)):
    y_predict[i] = predict(X_test[i],parameters,classes,order)

  correct = np.sum(y_predict == Y_test)
  print(correct, "are correct out of", len(y_predict),"test samples")
  print("Accuracy =",(correct*100)/len(y_predict),"%")

1e-9